## Delegierungs Setup - Draft

Dieses kleine Notebook soll helfen eine Delegierung aufzusetzen.
Benötigt werden in der Regel ja ggf. Aliase im DNS. 
Dann SPNs, welche einem Service Konto zugeordnet sind.
Am Ende benötigt dann ein Account die Delegiernug auf einen entsprechenden SPN.

### Aliase im DNS
Zuerst sollten die entsprechenden Variablen definiert werden, was so hier und da benötigt wird. Im unten stehenden Beispiel auf das -WhatIf achten.  Der Code wird dadurch NICHT ausgeführt.

In [ ]:
#'CNAME/Alias erstellen 
#'''''''''''''''''''''''''
# import-module dnsserver
<# Reminder
https://docs.microsoft.com/en-us/powershell/module/dnsserver/?view=win10-ps
DnsServer Module can be obtained either by installing DNS Server role or adding the DNS Server Tools part of Remote Server Administration Tools (RSAT) feature.
#>
$domain = 'domain.local'
$AliasName = 'SQL-MyAlias.domain.local'
$targetServer = 'MyTgtServer'
$target = $targetServer + '.' + $domain
$target

$DNSSrv = 'MyDNS.Domain.local'

Add-DnsServerResourceRecordCName -Name $AliasName -HostNameAlias $target -ZoneName $domain  -Verbose -ComputerName $DNSSrv  -WhatIf

Benötigen wir mehr als einen Alias, dann kann man vorab natürlich ein Array von aliasen definieren und dann die über eine ForEach Schleife anlegen. Wichtig ist aber, dass die Aliase den gleichen Ziel-Host als Target haben! 
Beispiele:

In [ ]:
#'CNAME/Alias erstellen - Loop Example
#'''''''''''''''''''''''''
$domain = 'domain.local'
$AliasNameArray = @("SMyFirstAlias","MySecondAlias")
$targetServer = 'MyTgtServer'
$target = $targetServer + '.' + $domain
#$target

$DNSSrv = 'MyDNS.Domain.local'

ForEach ($AliasName in $AliasNameArray){
Add-DnsServerResourceRecordCName -Name $AliasName -HostNameAlias $target -ZoneName $domain  -Verbose -ComputerName $DNSSrv  -WhatIf
}


In [ ]:
#CNAME /Alias  noch etwas komplizierter, aber dann wird es irgendwann echt too much :-)
$domain = 'domain.local'
$DNSSrv = 'MyDNS.Domain.local'
$arrayTargetEnvironment = @("P", "Q")
$arrayTargetServiceType = @("DB", "AS")
$targetServer = 'MyTgtServer'
$targetServiceType = 'My-SQL-AppAlias-'
foreach ($targetenv in $arrayTargetEnvironment) {
    $target = $targetServer + $targetenv + '.' + $domain
    #$target 
    foreach ($targetService in $arrayTargetServiceType) {
        #$target
        $AliasName = $targetServiceType + $targetService + '-' +$targetenv 
        #$AliasName
        Write-Host 'Schreibe '  $AliasName  ' mit Ziel auf '  $target
        Add-DnsServerResourceRecordCName -Name $AliasName -HostNameAlias $target -ZoneName $domain -Verbose -ComputerName $DNSSrv -WhatIf
    }
}


### SPN erzeugen
Sind die entsprechenden Aliase im DNS gesetzt, so muss ein Alias einem (g)MSA zugeordnet werden. Zwar kann man hier auch wieder mit einigen Variablen das ganze rel. dynamisch machen, aber Zwecks einfachem Nachverfolgen dann lieber ein paar manuelle Schritte mehr.
Wichtig ist, dass man immer das richtige Srv Kto und den richtigen Service adressiert.  Also ein DB Svc User bekommt ein MSSQL SPN.  Ein AS Svc User bekommt ein MSOLAP SPN.

Gelöst wird dies hier im Notebook mit 2 separaten Code Zellen. Einmal für **DB** und einmal für **AS**.

In [ ]:
#SPN SETZEN  DB Service
$MeinServiceKto = "MyServiceAccount"  
#Variablen fuer Alias, Environment, Dienst
$AliasApp = "MyApp"
$TargetEnv = "P"
$Svc = "MSSQLSvc"
$domain = 'domain.local'
$MeinServiceKonto = $MeinServiceKto + $TargetEnv
$MeinServiceKonto


$DBSPNs2Add = @()
$DBSPNs2Add += $svc+ '/SQL-'+$AliasApp+'-DB-'+$Targetenv+'.'+$domain+':1433'  #still a bit static
$DBSPNs2Add += $svc+ '/SQL-'+$AliasApp+'-DB-'+$Targetenv+'.'+$domain   #still a bit static
$DBSPNs2Add


$mySvcAcc = Get-ADServiceAccount -Identity $MeinServiceKonto
$mySvcAcc.DistinguishedName
#SPN hinzufügen
Set-ADObject -Identity $mySvcAcc -Add @{"ServicePrincipalName"=$DBSPNs2Add} -Verbose  -WhatIf

In [ ]:
#einmal gucken was gesetzt wurde
Get-ADServiceAccount -Identity "$MeinServiceKonto" -Properties * | Select-Object displayname -ExpandProperty serviceprincipalname | Format-Table

In [ ]:
#SPN SETZEN  SSAS Service
$MeinServiceKto = "MyServiceAccount"  
#Variablen fuer Alias, Environment, Dienst
$AliasApp = "MyApp"
$TargetEnv = "Q"
$Svc = "MSOLAPSvc.3"
$domain = 'domain.local'
$MeinServiceKonto = $MeinServiceKto + $TargetEnv
$MeinServiceKonto

$DBSPNs2Add = @()
$DBSPNs2Add += $svc+ '/SQL-'+$AliasApp+'-AS-'+$Targetenv  #still a bit static
$DBSPNs2Add += $svc+ '/SQL-'+$AliasApp+'-AS-'+$Targetenv+'.'+$domain #still a bit static
$DBSPNs2Add


$mySvcAcc = Get-ADServiceAccount -Identity $MeinServiceKonto
$mySvcAcc.DistinguishedName
#SPN hinzufügen
Set-ADObject -Identity $mySvcAcc -Add @{"ServicePrincipalName"=$DBSPNs2Add} -Verbose  -WhatIf

In [ ]:
#einmal gucken was gesetzt wurde
Get-ADServiceAccount -Identity "$MeinServiceKonto" -Properties * | Select-Object displayname -ExpandProperty serviceprincipalname | Format-Table

## Delegierung konfigurieren:

Für die Delegierung muss dem Konto, welches delegieren soll, mitgeteilt werden, wohin delegiert werden kann.
Vorab müssen also folgende infos gesammelt werden:
* involviertes ServiceKonto und/oder ComputerKonto
* Ziel SPNs

Die unten stehende Code Zellen fügt einem ausgewählten ServiceAccount die SPNs, welche man in dem Array pflegen kann, dem Attribut **AllowedToDelegateTo** zu.  
Die erste Zelle ist für ein **normales** Benutzerkonto, die zweite Zelle für einen **(group)ManagedServiceAccount**. Der Unterschied ist im **Set-** Befehl zu erkennen.
Die dritte Zelle ist zum Setzen einer Delegierung von einem **ComputerKonto** zu einem SPN.

In [ ]:
#Hilfszelle fuer SPN Strings bauen, sofern man die oben erstellten Variablen nicht weiter verwenden will:
$AliasApp = "MyApp"
$TargetEnv = "P"
$Svc = "MSOLAPSvc.3" #$Svc = "MSSQLSvc"
$domain = 'domain.local'


$ASSPNs2Add = @()
$ASSPNs2Add += $svc+ '/SQL-'+$AliasApp+'-AS-'+$Targetenv 
$ASSPNs2Add += $svc+ '/SQL-'+$AliasApp+'-AS-'+$Targetenv+'.'+$domain
$ASSPNs2Add

$Svc = "MSSQLSvc"  #ACHTUNG Service Switch

$DBSPNs2Add = @()
$DBSPNs2Add += $svc+ '/SQL-'+$AliasApp+'-DB-'+$Targetenv+'.'+$domain+':1433'  #still a bit static
$DBSPNs2Add += $svc+ '/SQL-'+$AliasApp+'-DB-'+$Targetenv+'.'+$domain
$DBSPNs2Add


In [ ]:
#Delegierung von einem NORMALEN ServiceKonto hin zu SPNs (Zelle 1)
#Import-Module ActiveDirectory 
$Account = "MyNormalSvcAccount"
$mySvcAcc = Get-ADUser -Identity $Account
#Was ist bisher gesetzt?
#Get-ADUser -Identity $Account  -Properties * |Select-Object displayname -ExpandProperty msDS-AllowedToDelegateTo | Format-Table
<#

$SPNS2AddDB = @()   #Hier müssen einmal die Ziel SPNs eingepflegt werden, ordentlich arbeiten  ;-) 
$SPNS2AddDB += "MSSQLSvc/blablabla.domain.local:1433","MSSQLSvc/blablabla.domain.local"
and so on
$SPNS2AddDB 
#>

$ASSPNs2Add  #either filled manually or pick the result from previous cell
$DBSPNs2Add

#Jetzt die 
#bei Bedarf kann die ein oder andere Zeile auskommentiert werden, je nachdem was für Aliase gesetzt werden sollenDelegierung setzen:
Set-ADObject -Identity $mySvcAcc -Add @{"msDS-AllowedToDelegateTo"=$ASSPNs2Add } -Verbose  -WhatIf  #ACHTUNG! Whatif aktiv, keine Eintraege
Set-ADObject -Identity $mySvcAcc -Add @{"msDS-AllowedToDelegateTo"=$DBSPNs2Add } -Verbose  -WhatIf  #ACHTUNG! Whatif aktiv, keine Eintraege



In [ ]:
#Kontrolle über gesetzte Delegierung
Get-ADUser -Identity $Account  -Properties * |Select-Object displayname -ExpandProperty msDS-AllowedToDelegateTo | Format-Table

In [ ]:
#Hier kommt noch der code für einen (group)ManagedServiceAccount hin !!!
Write-Host "Hier gibt es bisher nichts zu sehen"
#script will have same logic as last cell, but with different set- command

In [ ]:
# ComputerKonto Delegierung setzen
$comp2Edit = "MyComputer"

#$mycompt = Get-ADComputer $comp2Edit -Properties servicePrincipalName 
#$mycompt.servicePrincipalName

#Get-ADcomputer $comp2Edit  -Properties * |Select-Object displayname -ExpandProperty msDS-AllowedToDelegateTo | Format-Table
$ASSPNs2Add
$DBSPNs2Add
Set-ADComputer -Identity $comp2Edit -add @{"msDS-AllowedToDelegateTo"=$ASSPNs2Add} -Verbose  -WhatIf
#Set-ADComputer -Identity $comp2Edit -add @{"msDS-AllowedToDelegateTo"=$DBSPNs2Add} -Verbose  -WhatIf

In [ ]:
# Kontrolle über gesetzte Delegierung. Es kann sein, dass die neue Delegierung nicht sofort angeziegt wird. Zelle einfach 30 Sek später nochmal ausführen
Get-ADcomputer $comp2Edit  -Properties * |Select-Object displayname -ExpandProperty msDS-AllowedToDelegateTo | Format-Table